In [52]:
# Import packages
import pandas as pd
import numpy as np
import re
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import ast
from wordcloud import WordCloud
import matplotlib.colors as mcolors

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [33]:
# !python -m spacy download en_core_web_md

     |████████████████████████████████| 42.8 MB 542 kB/s eta 0:00:012
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# from sklearn.datasets import fetch_20newsgroups
# newsgroups_train = fetch_20newsgroups(subset='train')
# data = newsgroups_train.data

In [ ]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
# data_words = list(sent_to_words(data[:4]))
# print(data_words[:4])

### Project

In [2]:
df = pd.read_csv('data/philosophy_data.csv')

+ The dataset contains over 300,000 sentences from over 50 texts spanning 10 major schools of philosophy. The represented schools are: Plato, Aristotle, Rationalism, Empiricism, German Idealism, Communism, Capitalism, Phenomenology, Continental Philosophy, and Analytic Philosophy.

In [3]:
schools = ['plato', 'aristotle', 'empiricism', 'rationalism', 'analytic',
           'continental', 'phenomenology', 'german_idealism', 'communism',
           'capitalism', 'stoicism', 'nietzsche', 'feminism']

In [9]:
df

,title,author,school,sentence_spacy,sentence_str,original_publication_date,corpus_edition_date,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
0,Plato - Complete Works,Plato,plato,"What's new, Socrates, to make you leave your ...","What's new, Socrates, to make you leave your ...",-350,1997,125,"what's new, socrates, to make you leave your ...","['what', 'new', 'socrates', 'to', 'make', 'you...","what be new , Socrates , to make -PRON- lea..."
1,Plato - Complete Works,Plato,plato,Surely you are not prosecuting anyone before t...,Surely you are not prosecuting anyone before t...,-350,1997,69,surely you are not prosecuting anyone before t...,"['surely', 'you', 'are', 'not', 'prosecuting',...",surely -PRON- be not prosecute anyone before ...
2,Plato - Complete Works,Plato,plato,The Athenians do not call this a prosecution b...,The Athenians do not call this a prosecution b...,-350,1997,74,the athenians do not call this a prosecution b...,"['the', 'athenians', 'do', 'not', 'call', 'thi...",the Athenians do not call this a prosecution ...
3,Plato - Complete Works,Plato,plato,What is this you say?,What is this you say?,-350,1997,21,what is this you say?,"['what', 'is', 'this', 'you', 'say']",what be this -PRON- say ?
4,Plato - Complete Works,Plato,plato,"Someone must have indicted you, for you are no...","Someone must have indicted you, for you are no...",-350,1997,101,"someone must have indicted you, for you are no...","['someone', 'must', 'have', 'indicted', 'you',...","someone must have indict -PRON- , for -PRON- ..."
...,...,...,...,...,...,...,...,...,...,...,...
360803,"Women, Race, And Class",Davis,feminism,But the socialization of housework including m...,But the socialization of housework including m...,1981,1981,142,but the socialization of housework including m...,"['but', 'the', 'socialization', 'of', 'housewo...",but the socialization of housework include me...
360804,"Women, Race, And Class",Davis,feminism,The only significant steps toward endingdomest...,The only significant steps toward endingdomest...,1981,1981,117,the only significant steps toward endingdomest...,"['the', 'only', 'significant', 'steps', 'towar...",the only significant step toward endingdomest...
360805,"Women, Race, And Class",Davis,feminism,"Working women, therefore, have a special and v...","Working women, therefore, have a special and v...",1981,1981,90,"working women, therefore, have a special and v...","['working', 'women', 'therefore', 'have', 'spe...","working woman , therefore , have a special an..."
360806,"Women, Race, And Class",Davis,feminism,"Moreover, under capitalism, campaigns for jobs...","Moreover, under capitalism, campaigns for jobs...",1981,1981,199,"moreover, under capitalism, campaigns for jobs...","['moreover', 'under', 'capitalism', 'campaigns...","moreover , under capitalism , campaign for jo..."


In [8]:
# Distinct Schools
df['school'].unique()

array(['plato', 'aristotle', 'empiricism', 'rationalism', 'analytic',
       'continental', 'phenomenology', 'german_idealism', 'communism',
       'capitalism', 'stoicism', 'nietzsche', 'feminism'], dtype=object)

In [5]:
for sch in schools:
    print(f'{sch}:', df[df['school']==sch]['author'].unique())

plato: ['Plato']
aristotle: ['Aristotle']
empiricism: ['Locke' 'Hume' 'Berkeley']
rationalism: ['Spinoza' 'Leibniz' 'Descartes' 'Malebranche']
analytic: ['Russell' 'Moore' 'Wittgenstein' 'Lewis' 'Quine' 'Popper' 'Kripke']
continental: ['Foucault' 'Derrida' 'Deleuze']
phenomenology: ['Merleau-Ponty' 'Husserl' 'Heidegger']
german_idealism: ['Kant' 'Fichte' 'Hegel']
communism: ['Marx' 'Lenin']
capitalism: ['Smith' 'Ricardo' 'Keynes']
stoicism: ['Epictetus' 'Marcus Aurelius']
nietzsche: ['Nietzsche']
feminism: ['Wollstonecraft' 'Beauvoir' 'Davis']


In [6]:
for sch in schools:
    print(f'{sch}:',df[df['school']==sch]['original_publication_date'].unique())

plato: [-350]
aristotle: [-320]
empiricism: [1689 1739 1779 1713 1710]
rationalism: [1677 1710 1637 1641 1674]
analytic: [1921 1912 1910 1953 1985 1950 1959 1972 1975]
continental: [1963 1961 1966 1967 1968 1972]
phenomenology: [1945 1936 1907 1927 1950]
german_idealism: [1788 1790 1781 1798 1817 1807 1820]
communism: [1883 1848 1862]
capitalism: [1776 1817 1936]
stoicism: [125 170]
nietzsche: [1888 1886 1887]
feminism: [1792 1949 1981]


In [ ]:
for sch in schools:
    print(f'{sch}:',df[df['school']==sch]['title'].unique())
    print('')

In [4]:
df['tokenized_txt']

0         ['what', 'new', 'socrates', 'to', 'make', 'you...
1         ['surely', 'you', 'are', 'not', 'prosecuting',...
2         ['the', 'athenians', 'do', 'not', 'call', 'thi...
3                      ['what', 'is', 'this', 'you', 'say']
4         ['someone', 'must', 'have', 'indicted', 'you',...
                                ...                        
360803    ['but', 'the', 'socialization', 'of', 'housewo...
360804    ['the', 'only', 'significant', 'steps', 'towar...
360805    ['working', 'women', 'therefore', 'have', 'spe...
360806    ['moreover', 'under', 'capitalism', 'campaigns...
360807    ['this', 'strategy', 'calls', 'into', 'questio...
Name: tokenized_txt, Length: 360808, dtype: object

In [ ]:
### 

### School: Capitalism & Author: Keynes

In [24]:
truncated_ds = df[(df['school']=='capitalism') & (df['author']=='Keynes')]['tokenized_txt']
data_words = []
for key, value in truncated_ds.items():
    value = ast.literal_eval(value)
    data_words.append(value)

In [25]:
print(data_words[0:4])

[['capitalism', 'is', 'not', 'for', 'the', 'faint', 'of', 'heart'], ['it', 'is', 'system', 'of', 'supply', 'and', 'demand', 'that', 'reduces', 'real', 'workingmen', 'and', 'workingwomen', 'into', 'graphs', 'and', 'equations', 'subject', 'to', 'aggregate', 'observations', 'devoid', 'of', 'any', 'real', 'human', 'factors'], ['if', 'left', 'to', 'regulate', 'itself', 'the', 'economy', 'should', 'remain', 'in', 'check', 'and', 'avoid', 'dangerously', 'radical', 'changes', 'in', 'productivity', 'orthodox', 'economists', 'maintain'], ['how', 'then', 'do', 'we', 'explain', 'terrible', 'recessions', 'such', 'as', 'the', 'great', 'depression', 'where', 'unemployment', 'figures', 'were', 'seen', 'as', 'high', 'as', 'with', 'still', 'more', 'underemployed', 'and', 'working', 'far', 'below', 'their', 'experience', 'and', 'capability']]


In [66]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [28]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [29]:
data_words_nostops = remove_stopwords(data_words)

In [53]:
print(data_words_nostops[4:8])

[['system', 'corrected', 'dire', 'circumstances', 'created'], ['economists', 'reply', 'simply', 'workers', 'unwilling', 'accept', 'lower', 'wages', 'times', 'decline', 'would', 'rather', 'quit', 'thus', 'jeopardizing', 'beautifully', 'constructed', 'apparently', 'fragile', 'classical', 'theory', 'economics'], ['arguments', 'effective', 'always', 'fallback', 'plan', 'declaring', 'social', 'darwinism', 'great', 'depression', 'serving', 'perfect', 'opportunity', 'weed', 'worst', 'employees', 'best', 'would', 'emerge', 'victorious', 'unforeseeable', 'future', 'date'], ['first', 'months', 'following', 'explosion', 'depressed', 'economic', 'data', 'perhaps', 'population', 'would', 'nervously', 'accept', 'postulates']]


In [57]:
print(data_words_bigrams[0:4])

[['capitalism', 'faint', 'heart'], ['system', 'supply', 'demand', 'reduces', 'real', 'workingmen', 'workingwomen', 'graphs', 'equations', 'aggregate', 'observations', 'devoid', 'real', 'human', 'factors'], ['left', 'regulate', 'economy', 'remain', 'check', 'avoid', 'dangerously', 'radical', 'changes', 'productivity', 'orthodox', 'economists', 'maintain'], ['explain', 'terrible', 'recessions', 'great', 'depression', 'unemployment', 'figures', 'seen', 'high', 'still', 'underemployed', 'working', 'far', 'experience', 'capability']]


In [65]:
print(bigram_mod[data_words_bigrams[5]])

['economists', 'reply', 'simply', 'workers', 'unwilling', 'accept', 'lower', 'wages', 'times', 'decline', 'would', 'rather', 'quit', 'thus', 'jeopardizing', 'beautifully', 'constructed', 'apparently', 'fragile', 'classical', 'theory', 'economics']


In [36]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [37]:
print(data_lemmatized[0:4])

[['capitalism', 'faint', 'heart'], ['system', 'supply', 'demand', 'reduce', 'real', 'workingmen', 'graph', 'equation', 'aggregate', 'observation', 'devoid', 'real', 'human', 'factor'], ['leave', 'regulate', 'economy', 'remain', 'check', 'avoid', 'dangerously', 'radical', 'change', 'productivity', 'orthodox', 'economist', 'maintain'], ['explain', 'terrible', 'recession', 'great', 'depression', 'unemployment', 'figure', 'see', 'high', 'still', 'underemploye', 'work', 'far', 'experience', 'capability']]


In [38]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized

In [39]:
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:4]) #it will print the corpus we created above.

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1)], [(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)], [(29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1)]]


In [40]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, random_state=100, 
                                            update_every=1, chunksize=100, passes=10, alpha='auto',
                                            per_word_topics=True)

In [44]:
lda_model.get_topics()

array([[6.4954620e-05, 6.4954089e-05, 6.4954089e-05, ..., 6.4962129e-05,
        6.4983193e-05, 6.4983193e-05],
       [1.1823566e-04, 1.1823516e-04, 1.1823516e-04, ..., 1.1823519e-04,
        1.1823517e-04, 1.1823517e-04],
       [5.3729622e-05, 5.3727828e-05, 5.3727828e-05, ..., 5.3743533e-05,
        5.3732667e-05, 5.3732667e-05],
       ...,
       [2.3122304e-05, 2.3121420e-05, 2.3121420e-05, ..., 2.3133098e-05,
        2.3128347e-05, 2.3128347e-05],
       [2.1653441e-05, 2.1651804e-05, 2.1651804e-05, ..., 2.1661515e-05,
        2.1654796e-05, 2.1654796e-05],
       [8.1381055e-05, 8.1380676e-05, 8.1380676e-05, ..., 8.1386570e-05,
        8.1473947e-05, 8.1473947e-05]], dtype=float32)

In [43]:
pprint(lda_model.print_topics())

[(0,
  '0.266*"time" + 0.127*"indeed" + 0.073*"conclusion" + 0.073*"standard" + '
  '0.062*"analysis" + 0.059*"become" + 0.055*"profit" + 0.012*"derive" + '
  '0.000*"increase" + 0.000*"investment"'),
 (1,
  '0.212*"determine" + 0.178*"right" + 0.054*"other" + 0.052*"alternative" + '
  '0.006*"third" + 0.000*"political" + 0.000*"wrong" + 0.000*"philosopher" + '
  '0.000*"mood" + 0.000*"investment"'),
 (2,
  '0.153*"term" + 0.115*"effect" + 0.109*"different" + 0.107*"policy" + '
  '0.048*"short" + 0.037*"spend" + 0.031*"discussion" + 0.028*"economy" + '
  '0.027*"government" + 0.025*"use"'),
 (3,
  '0.137*"change" + 0.092*"level" + 0.090*"condition" + 0.081*"fall" + '
  '0.066*"certain" + 0.061*"country" + 0.058*"proportion" + 0.048*"force" + '
  '0.044*"assume" + 0.036*"show"'),
 (4,
  '0.288*"interest" + 0.251*"rate" + 0.077*"price" + 0.063*"consumption" + '
  '0.034*"part" + 0.025*"play" + 0.020*"book" + 0.020*"saving" + '
  '0.019*"necessary" + 0.019*"dispute"'),
 (5,
  '0.409*"wage

In [49]:
def format_topics_sentences(ldamodel=None, corpus=corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    return sent_topics_df

    # Add original text to the end of the output
#     contents = pd.Series(texts)
#     sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
#     return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords']
df_dominant_topic.head(20)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords
0,0,4.0,0.1106,"interest, rate, price, consumption, part, play..."
1,1,9.0,0.4118,"idea, volume, real, demand, factor, aggregate,..."
2,2,16.0,0.2398,"influence, therefore, field, correct, particul..."
3,3,18.0,0.4413,"state, great, production, measure, well, exist..."
4,4,16.0,0.1572,"influence, therefore, field, correct, particul..."
5,5,11.0,0.2183,"full, low, matter, rather, decline, teach, wor..."
6,6,9.0,0.2959,"idea, volume, real, demand, factor, aggregate,..."
7,7,9.0,0.1858,"idea, volume, real, demand, factor, aggregate,..."
8,8,10.0,0.3236,"even, value, event, people, contemporary, rece..."
9,9,18.0,0.2936,"state, great, production, measure, well, exist..."
